In [1]:
# Install the unsloth package from the GitHub repository
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: unsloth 2024.12.4
Uninstalling unsloth-2024.12.4:
  Successfully uninstalled unsloth-2024.12.4
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-w8_tybyc/unsloth_50b0c9ce64974e3e9b6eaea8abdd31e8
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-w8_tybyc/unsloth_50b0c9ce64974e3e9b6eaea8abdd31e8
  Resolved https://github.com/unslothai/unsloth.git to commit 85f1fa096afde5efe2fb8521d8ceec8d13a00715
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.4-py3-none-any.whl size=173746 sha256=94c18fbf9fcccc21f11f117cc77dccd8de9a31a6a3c1d6f981b37ffb981c69e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-xonctfap/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth


### Load the necessary packages

In [2]:

import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

/home/fozle/Projects/Personalized-LLM/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### Load the base model from unsloth and prepare PEFT parameters

In [7]:


max_seq_length = 5020

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

# Prepare training with peft
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 32,
    loftq_config = None,
)
print(model.print_trainable_parameters())

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.61 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.12.4 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039
None


### Prepare training prompts

In [8]:
data_prompt = """Analyze the provided text from a mental health perspective. Identify any indicators of emotional distress, coping mechanisms, or psychological well-being. Highlight any potential concerns or positive aspects related to mental health, and provide a brief explanation for each observation.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(examples):
    inputs       = examples["Context"]
    outputs      = examples["Response"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


In [9]:
import pandas as pd

In [10]:
#Format the data for training

# Load the CSV data
filtered_data = pd.read_csv('./dataset/train_mental_health.csv')

training_data = Dataset.from_pandas(filtered_data)
training_data = training_data.map(formatting_prompt, batched=True)


Map: 100%|██████████| 2784/2784 [00:00<00:00, 147116.64 examples/s]


In [11]:
training_data[0]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someh

### Model Training 🤖

In [12]:
#Training Process
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=training_data,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=32,
        gradient_accumulation_steps=8,
        num_train_epochs=40,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
    ),
)

trainer.train()

Generating train split: 182 examples [00:00, 765.41 examples/s]
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 182 | Num Epochs = 40
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 8
\        /    Total batch size = 256 | Total steps = 40
 "-____-"     Number of trainable parameters = 11,272,192
  2%|▎         | 1/40 [01:30<58:31, 90.04s/it]

{'loss': 2.3746, 'grad_norm': 1.6996452808380127, 'learning_rate': 2.9999999999999997e-05, 'epoch': 1.0}


  5%|▌         | 2/40 [02:55<55:26, 87.55s/it]

{'loss': 2.3746, 'grad_norm': 1.699662446975708, 'learning_rate': 5.9999999999999995e-05, 'epoch': 2.0}


  8%|▊         | 3/40 [04:21<53:22, 86.54s/it]

...

100%|██████████| 40/40 [56:43<00:00, 84.72s/it]

{'loss': 1.8213, 'grad_norm': 0.11374007910490036, 'learning_rate': 0.0, 'epoch': 40.0}


100%|██████████| 40/40 [56:47<00:00, 85.18s/it]

{'train_runtime': 3407.0003, 'train_samples_per_second': 2.137, 'train_steps_per_second': 0.012, 'train_loss': 2.0181788563728333, 'epoch': 40.0}


TrainOutput(global_step=40, training_loss=2.0181788563728333, metrics={'train_runtime': 3407.0003, 'train_samples_per_second': 2.137, 'train_steps_per_second': 0.012, 'total_flos': 2.158569454239744e+17, 'train_loss': 2.0181788563728333, 'epoch': 40.0})

In [13]:
# Saving the model locally
model.save_pretrained("ft_model/1B_finetuned_llama3.2")
tokenizer.save_pretrained("ft_model/1B_finetuned_llama3.2")

('ft_model/1B_finetuned_llama3.2/tokenizer_config.json',
 'ft_model/1B_finetuned_llama3.2/special_tokens_map.json',
 'ft_model/1B_finetuned_llama3.2/tokenizer.json')

### Load the fine-tuned model

In [14]:
model, tokenizer = FastLanguageModel.from_pretrained(
model_name = "ft_model/1B_finetuned_llama3.2",
max_seq_length = 5020,
dtype = None,
load_in_4bit = True)
model = FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.61 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


#### Inference

In [15]:
import gc
# Clear CUDA cache
def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

In [17]:
def generate_response(model, tokenizer, text, data_prompt):
    clear_memory()
    
    with torch.no_grad():
        inputs = tokenizer(
            [data_prompt.format(input=text)],
            return_tensors="pt",
            padding=True
        ).to("cuda")
        
        outputs = model.generate(
            **inputs,
            max_new_tokens=5020,  # Keeping the original parameter
            use_cache=True,
            do_sample=True,
            num_return_sequences=1,
            # streamer=TextStreamer(tokenizer)
        )
        
        response = tokenizer.batch_decode(outputs)[0]
        cleaned_response = response.split("Response:")[-1].strip()
        
        # Remove unwanted tokens
        cleaned_response = cleaned_response.replace("<|begin_of_text|>", "").replace("<|end_of_text|>", "").strip()
        
    clear_memory()
    return cleaned_response

In [18]:
data_prompt = """Analyze the mental health aspects in this text:
{input}
Response:"""

In [19]:
text = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"

response = generate_response(model, tokenizer, text, data_prompt)
response

"It sounds like there is more going on emotionally than you're aware of. A good therapist will help you explore what's going on for you and help you find ways to express your feelings. My guess is that you're afraid people won't accept you as you are, or that they will reject you. \xa0Are there times when you wonder if people would even care about what you're feeling? Often when people are depressed they feel alone or think no one cares. \xa0You seem to be aware that you're worth something and believe you're worthwhile. The good news is that it sounds like you have a therapist now (congratulations!) and that you know you need help. Now...to get what you need might require some changes to your life. Do you have a good support system? What are you doing that helps you not despair? A therapist can also help you to see what's at the root of this emotion. You may have some unhealthy habits or things you are telling yourself that are not helpful and may need to change. Getting support from o